# Assignment 2


In this assignment we will cover topics from the previous 3 lectures. We will cover the following topics:

1) Training a simple Linear Model

2) Implementing Modules with Backprop functionality

3) Implementing Convolution Module on Numpy.

4) Implement Dropout/Different Optimizer setups.

5) Implementing Pool and Training on CIFAR10?


It is crucial to get down to the nitty gritty of the code to implement all of these. No external packages (like caffe,pytorch etc), which directly give functions for these steps, are to be used. 

# Training a simple Linear Model

In this section, you will write the code to train a Linear Model. The goal is to classify and input $x_n$ of size $n$ into one of $m$ classes. For this goal, you need to create the following parts:

1) ** A weight Matrix $W_{n\times m}$ **, where the Weights are multipled to the input $X_n$ (Vector of size $n$), to find $m$ scores $S_m$ for the $m$ classes.

2) ** The Loss function **: We learnt two Kinds of Loss functions:
  *  The Hinge Loss: This loss measures, for each sample, how many times were the wrong classes scored above correct class score - $\Delta$ ? and by how much? This leads to the formulation:
  
$$
L_i = \sum_{j\neq y_i} \max(0, s_j - s_{y_i} + \Delta)
$$

where $y_i$ is the correct class, and $s_j$ is the score for the $j$-th class (the $j$-th element of $S_m$)
  
  * The softmax Loss: By interpreting the scores as unnormalized log probabilities for each class, this loss tries to measure dissatisfaction with the scores in terms of the log probability of the right class:

$$
L_i = -\log\left(\frac{e^{f_{y_i}}}{ \sum_j e^{f_j} }\right) \hspace{0.5in} \text{or equivalently} \hspace{0.5in} L_i = -f_{y_i} + \log\sum_j e^{f_j}
$$

where $f_{ y_i }$ is the $i$-th element of the output of $W^T_{n \times m} . X_m$

4) ** Regularization term **: In addition to the loss, you need a Regularization term to lead to a more distributed( in case of $L_2$) or sparse (in case of $L_1$) learning of the weights. For example, and having $L_2$ regularization would imply that your loss has the following additional term:

$$
R(W) = \sum_k\sum_l W_{k,l}^2,
$$

making the total loss:
$$
L =  \underbrace{ \frac{1}{N} \sum_i L_i }_\text{data loss} + \underbrace{ \lambda R(W) }_\text{regularization loss} \\\\
$$

3) ** An Optimization Procedure **: This refers to the process which tweaks the weight Matrix $W_{n\times m}$ to reduce the loss function $L$. In our case, this refers to Mini-batch Gradient Descent algorithm. We adjust the weights $W_{n\times m}$, based on the gradient of the loss $L$ w.r.t. $W_{n\times m}$. This leads to:

$$
W_{t+1} = W_{t} + \alpha \frac{\partial L}{\partial W},
$$
where $\alpha$ is the learning rate. Additionally, as we will be doing "mini-batch" gradient Descent, instead of finding loss over the whole dataset, we find it only for a small sample of the traning data for each learning step we take. Basically,
$$
W_{t+1} = W_{t} + \alpha \frac{\partial \sum^{b}{L_{x_i}}}{\partial W},
$$
where, $b$ is the batch size.

# Question 1

Train a **Single-Layer Classifier** for the MNIST dataset. Guidelines:
* Use a loss of your choice.
* Keep a validation split of the trainingset for finding the right value of $\lambda$ for the regularization, and to check for over fitting.
* Finally,evaluate the classification performance on the testset.


In [ ]:
## Load The Mnist data:
# Download data from http://yann.lecun.com/exdb/mnist/
# load the data.

# split the data into train, and valid

# Now a function, which returns a generator random mini-batch of the input data

def get_minibatch_function(training_x, training_y):
    
    def get_minibatch(training_x=training_x, training_y=training_y):
        ## Read generator functions if required.

        ## WRITE CODE HERE
        
        yield mini_x,mini_y
        
    return get_minibatch

In [ ]:
# Define the class Single Layer Classifier
class single_layer_classifier():
    
    def __init__(self, input_size, output_size):
        
        ## WRITE CODE HERE
        
        # Give the instance a weight matrix, initialized randomly.
        
    # Define the forward function
    def forward(self, input_x):
        
        # get the scores
        
        ## WRITE CODE HERE
        
        return scores
    
    # Similarly a backward function
    # we define 2 backward functions (as Loss = L1 + L2, grad(Loss) = grad(L1) + grad(L2))
    
    def backward_from_loss(self, grad_from_loss):
        
        # this function returns a matrix of the same size as the weights, 
        # where each element is the partial derivative of the loss w.r.t. the respective element of weight.
        
        ## WRITE CODE HERE
        
        return grad_matrix
        
    def backward_from_l2(self):
        
        # this function returns a matrix of the same size as the weights, 
        # where each element is the partial derivative of the regularization_term
        # w.r.t. the respective element of weight.
        
        ## WRITE CODE HERE
        
        return grad_matrix
    
    # BONUS
    def grad_checker(input_x, grad_matrix):
        
        # Guess what to do?
        
        ## WRITE CODE HERE
        
        if diff<threshold:
            return true
        else:
            return false

In [ ]:
# Now we need the loss functions,one which calculates the loss, 
# and one which give the backward gradient
# Make any one of the suggested losses

def loss_forward(input_y,scores):

    ## WRITE CODE HERE    
    
    return loss
def loss_backward(loss):
    # This part deals with the gradient from the loss to the weight matrix.
    # for example, in case of softmax loss(-log(qc)), this part gives grad(loss) w.r.t. qc

    ## WRITE CODE HERE    

    return grad_from_loss
        

In [ ]:
# Finally the trainer:

# let it be for t iterations:

# make an instance of single_layer_classifier,
# get the mini-batch yielder.

for iter,input_x, input_y in enumerate(get_minibatch()):
    
    ## Write code here for each iteration of training.

In [ ]:
# Find the performance on the validation set.
# find the top-1 accuracy on the validation set.

In [ ]:
# now make a trainer function based on the above code, which trains for 't' iteration,
# and returns the performance on the validation

def trainer(iterations, kwargs):

    ## WRITE CODE HERE
    
    return top_1


In [ ]:
# Find the optimal lambda and iterations t

In [ ]:
# Train on whole dataset with these values,(from scratch)
# report final performance on mnist test set.

# Find the best performing class and the worst performing class.


# Implementing Backprop

Now that you have had some experience with single layer networks, its time to go to more complex architectures. But first we need to completely understand and implement backpropagation.

## Backpropagation:

Simple put, a way of computing gradients of expressions through recursive application of chain rule. If,
$$
L = f (g (h (\textbf{x})))
$$
then,
$$
\frac{\partial L}{\partial \textbf{x}} = \frac{\partial f}{\partial g} \times \frac{\partial g}{\partial h} \times\frac{\partial h}{\partial \textbf{x}} 
$$

** Look into the class Lecture for more detail **



# Question 2 : Scalar Backpropagation

Evaluate the gradient of the following functions w.r.t. the input.

1) $$ f(x,y,z) =  log(\sigma(\frac{cos(\pi \times \sigma(x))+sin(\pi \times \sigma(y/2))}{z^2}))$$
where $\sigma$ is the sigmoid function. Find gradient for the following values:
  * $(x,y,z)$ =  (1,2,3)
  * $(x,y,z)$ =  (3,2,1)
  * $(x,y,z)$ =  (12,23,31)
  * $(x,y,z)$ =  (32,21,13)

2) $$ f(x,y,z) = -tan(z) + exp(4x^2 + 3x + 10) - x^{y^z} $$
where $\exp$ is the exponential function. Find gradient for the following values:
  * $(x,y,z)$ =  (-0.1 ,2 ,-3)
  * $(x,y,z)$ =  (-3, 0.2,0.5)
  * $(x,y,z)$ =  (1.2, -2.3, 3.1)
  * $(x,y,z)$ =  (3.2, 2.1, -1.3)
      

In [ ]:
# To solve this problem, construct the computational graph (will help understanding the problem)(not part of assignment)
# Write each component of the graph as a class, with forward and backward functions.

# for eg:
class sigmoid():
    def __init__(self):
        
    def forward():
        # save values useful for backpropagation
    def backward():
        
# CAUTION: Carefully treat the input and output dimension variation. At worst, handle them with if statements.
# Similarly create the classes for various sub-parts/elements of the graph.

In [ ]:
# Now write func_1_creator, 
# which constructs the graph(all operators), forward and backward functions.

class func1():
    def __init__(self):
        # construct the graph here, 
        # assign the instances of function modules to self.var
        
    def forward(x,y,z):
        # Using the graph element's forward functions, get the output. 
        
        return output
    
    def backward():
        # Use the saved outputs of each module, and backward() function calls
        
        return [grad_x,grad_y,grad_z]
    
    
# Similarly,
class func2():
    def __init__(self):
        # construct the graph here, 
        # assign the instances of function modules to self.var
        
    def forward(x,y,z):
        # Using the graph element's forward functions, get the output. 
        
        return output
    
    def backward():
        # Use the saved outputs of each module, and backward() function calls
        
        return [grad_x,grad_y,grad_z]

## Question 3 : Modular Vector Backpropagation

* Construct a Linear Layer module, implementing the forward and backward functions for arbitrary sizes.
* Construct a ReLU module, implementing the forward and backward functions for arbitrary sizes.
* Create a 2 layer MLP using the constructed modules.

* Modifying the functions built in Question 1 , train this two layer MLP for the same data set (MNIST).

In [ ]:
# Class for Linear Layer (Refer code of pytorch/tensorflow package if required.) 


In [ ]:
# Class for ReLU


In [ ]:
# Your 2 layer MLP 


In [ ]:
# Train the MLP


In [ ]:
# Validation Performance


In [ ]:
# Best Class and worst class performance.


# After the lecture on Jan 31st.

# Implementing Convolution Module on Numpy.

* This topic will require you to implement the Convolution operation using Numpy.
* You will implement <s>two</s> one methods of doing it, an intuitive <s>and an optimised</s> way.
* <s>Additional operations like dropout, batch norms.</s>
* We will use the created Module for interesting task like Blurring, Bilateral Filtering.
* Finally, we create the Backprop for this.
* <s>Train a Conv model for the same MNIST dataset. (can be a script based training, instead of having it in jupyter notebook.)</s>


## Question 4

<br>
* Implement a naive Convolution module, with basic functionalities:
  * kernel_size,padding, stride, dilation
  
* Test out the convolution layer created, by using it to do gaussian blurring on 10 random images of Cifar10 dataset

* Bonus: Bilateral filtering can also be implemented using a 2-D convolution. Try bilateral filter for the space of (X,Y,Gray). (3D space, but not 3D conv), (no speed criteria), (Hint: You have multiple filters in each conv layer.)


In [ ]:
## Define a class Convolution Layer, which is initialized with the various required params:
Class convolution_layer():
    
    def __init__(self,kvargs):
        ## Refer pytorch documentation/tensorflow documentation for the parameters for the layer.
        ## Bonus for implementing Groups, no-bias functionality.
        ## Random initialization of the weights
        
    def forward(self,input):
        # Input Proprocess(According to pad etc.) Input will be of size (Batch_size, in_channels, inp_height, inp_width)
        
        # Reminder: Save Input for backward!
        # Simple Conv operation:
        # Loop over every location in inp_height * inp_width for the whole batch
        
        # Output will be of the size (Batch_size, out_channels, out_height, out_width)
        return output
    
    def backward(self, grad_of_output_size):
        
        # Naive Implementation
        # Speed is not a concern
        # Hint: gradients from each independant operation can be summed
        
        #  return gradient of the size of the weight kernel
        return grad
    
    def set_weights(self, new_weights):
        ## Replace the set of weights with the given 'new_weights'
        ## use this for setting weights for blurring, bilateral filtering etc. 
    

In [ ]:
## get cifar images

## Initialize a conv layer. Set weights for gaussian blurring

## generate output.

## use matplotlib.pyplot to show the results.


In [ ]:
## BONUS: Bilateral Filter.


## Question 5
<br>
Now we will use the created layer for training a simple Convolution Layer. 

* The goal is to make it learn a set of weights, by using the backpropagation function created. To test the backpropagation, instead of training a whole network, we will train only a single layer.
  * Take 100 cifar10 images. Generate a numpy array of size (20,3,5,5), with samples from uniform distribution (-1,1).Initialize a Convolution layer with 20 5$\times$5 kernels(input size 3) and set the generated weights as the layer weights. Save the output of these 100 images from this Convolution layer. 
  
  * Now, initialize a new convolution layer, and use $L_2$ loss between the output of the network and the output generated in the previous step to get the same set of weights as the ones generated in the previous step.


In [ ]:
## First generate the random weight vector

## Init a conv layer with these weights

## For all images get output. Store in numpy array.



In [ ]:
# for part 2 we need to write a small L2 layer
class L2_loss():
    def ___init__():
        # empty
    
    def forward(self, inp_1,inp_2):
        # input is of dimestion(batch,channels,h,w)
        # calculate the l2 norm of inp_1 - inp_2 .,
        
        return output
    
    def backward(self,output_grad):
        # from the loss, and the input, get the grad at each location of the input.
        # The grad is of the shape (batch,channels,h,w)
        return grad

# Now Init a new conv layer and a L2 loss layer

# Train the new conv-layer using the L2 loss to get the earlier set of generated weights.
# Use batches.


# Print L2 dist between output from new convolution layer and the outputs generated initially.
